In [120]:
# make sure to install these packages before running:
!pip install pandas
!pip install sodapy
!pip install python-dotenv

In [121]:
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv
import os
import json
import hashlib

## Data Collection and Acquisition

In [122]:
#load secrets

load_dotenv(override = True)

api_token = os.getenv("API_TOKEN")
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

In [123]:
#API authentication for the datasets
client = Socrata("data.ny.gov",
                 api_token,
                 username= email,
                 password= password)

**Dataset 1**

In [124]:
# returns json response - DATASET 1 “MTA Subway Trains Delayed: Beginning 2020”
results = client.get_all("g937-7k7c", 
    where="month >= '2024-01' AND month <= '2024-12'")

In [125]:
# Convert to pandas DataFrame - DATASET 1 “MTA Subway Trains Delayed: Beginning 2020”
delayed_df = pd.DataFrame.from_records(results)
delayed_df

,month,division,line,day_type,incidents,reporting_category
0,2024-01-01T00:00:00.000,A DIVISION,1,1,0,NaN
1,2024-01-01T00:00:00.000,A DIVISION,1,1,31,Crew Availability
2,2024-01-01T00:00:00.000,A DIVISION,1,1,2,External Factors
3,2024-01-01T00:00:00.000,A DIVISION,1,1,92,Infrastructure & Equipment
4,2024-01-01T00:00:00.000,A DIVISION,1,1,86,Operating Conditions
...,...,...,...,...,...,...
3761,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,16,External Factors
3762,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,6,Infrastructure & Equipment
3763,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Operating Conditions
3764,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Planned ROW Work


**Folder creation**

In [126]:
if not os.path.exists("raw_data"):
    os.mkdir("raw_data")

if not os.path.exists("SHA256"):
    os.mkdir("SHA256")

**Dataset 1 - SHA256 Integrity Check**

In [127]:
# Write the data to disk as a CSV
delayed_df.to_csv("raw_data/delayed.csv", index=False)

# Read the CSV as bytes
with open("raw_data/delayed.csv", "rb") as f:
    data_bytes = f.read()

# Calculate SHA-256 of uncompressed data
sha256_hash = hashlib.sha256(data_bytes).hexdigest()

# Write the hex digest
with open("SHA256/delayed.sha", "w") as f:
    f.write(sha256_hash)

**Dataset 1 - Cleaning and Quality Assessment BEFORE INTEGRATION**

In [128]:
#Drop NaN Values
delayed_df = delayed_df.dropna()
delayed_df

,month,division,line,day_type,incidents,reporting_category
1,2024-01-01T00:00:00.000,A DIVISION,1,1,31,Crew Availability
2,2024-01-01T00:00:00.000,A DIVISION,1,1,2,External Factors
3,2024-01-01T00:00:00.000,A DIVISION,1,1,92,Infrastructure & Equipment
4,2024-01-01T00:00:00.000,A DIVISION,1,1,86,Operating Conditions
5,2024-01-01T00:00:00.000,A DIVISION,1,1,49,Planned ROW Work
...,...,...,...,...,...,...
3761,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,16,External Factors
3762,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,6,Infrastructure & Equipment
3763,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Operating Conditions
3764,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Planned ROW Work


In [129]:
#Check Data Types -- originally all were objects! 
delayed_df.dtypes

month                 object
division              object
line                  object
day_type              object
incidents             object
reporting_category    object
dtype: object

In [130]:
#Convert to proper data types
delayed_df['month'] = pd.to_datetime(delayed_df['month']).dt.normalize()
delayed_df['division'] = delayed_df['division'].astype(str)
delayed_df['line'] = delayed_df['line'].astype(str)
delayed_df['day_type'] = delayed_df['day_type'].astype(str)
delayed_df['incidents'] = delayed_df['incidents'].astype(int)
delayed_df['reporting_category'] = delayed_df['reporting_category'].astype(str)

C:\Users\victo\AppData\Local\Temp\ipykernel_34608\3324790677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delayed_df['month'] = pd.to_datetime(delayed_df['month']).dt.normalize()
C:\Users\victo\AppData\Local\Temp\ipykernel_34608\3324790677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delayed_df['division'] = delayed_df['division'].astype(str)
C:\Users\victo\AppData\Local\Temp\ipykernel_34608\3324790677.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [131]:
#Clean string columns for synactic issues (capitalization and white space from front and back)
delayed_df['division'] = delayed_df['division'].str.strip().str.upper()
delayed_df['line'] = delayed_df['line'].str.strip().str.upper()
delayed_df['day_type'] = delayed_df['day_type'].str.strip().str.upper()
delayed_df['reporting_category'] = delayed_df['reporting_category'].str.strip().str.upper()

C:\Users\victo\AppData\Local\Temp\ipykernel_34608\828929010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delayed_df['division'] = delayed_df['division'].str.strip().str.upper()
C:\Users\victo\AppData\Local\Temp\ipykernel_34608\828929010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delayed_df['line'] = delayed_df['line'].str.strip().str.upper()
C:\Users\victo\AppData\Local\Temp\ipykernel_34608\828929010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [132]:
#Semantic check. Checking no numbers below 0 for incidents column. 
delayed_df[delayed_df['incidents'] < 0]

,month,division,line,day_type,incidents,reporting_category


In [133]:
#Syntactic check, make sure all the months are formatted the same.
delayed_df['month'].dt.strftime('%Y-%m-%d').unique()

array(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
       '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
       '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
      dtype=object)

In [134]:
#Semantic check. Make sure all Division A are associated with numbered subway lines and 
#all Division B are associated with lettered subway lines

bad_A = delayed_df[
    (delayed_df['division'] == 'A DIVISION') &
    ~delayed_df['line'].str.contains(r'\d', na=False)      
]
bad_B = delayed_df[
    (delayed_df['division'] == 'B DIVISION') &
    ~delayed_df['line'].str.fullmatch(r'[A-Z ]+')  
]

print("A DIVISION with non-numbered lines:")
print(bad_A)

print("\nB DIVISION with non-lettered lines:")
print(bad_B)

A DIVISION with non-numbered lines:
Empty DataFrame
Columns: [month, division, line, day_type, incidents, reporting_category]
Index: []

B DIVISION with non-lettered lines:
Empty DataFrame
Columns: [month, division, line, day_type, incidents, reporting_category]
Index: []


In [135]:
#Check for duplicate rows across the entire dataset
delayed_df[delayed_df.duplicated()]

,month,division,line,day_type,incidents,reporting_category


In [136]:
# checking for syntactic and semantic errors syntax issues
print("Lines:", delayed_df['line'].unique())
print("Divisions:", delayed_df['division'].unique())
print("Day Types:", delayed_df['day_type'].unique())
print("Reporting Categories:", delayed_df['reporting_category'].unique())

# so only going to do analysis on lines: 1', '2', '3', '4', '5', '6', '7', 'S 42ND', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'JZ', 'L', 'M', 'N', 'Q', 'R', 'S FKLN', 'S ROCK' ONLY
#because all these lines can be found in both datasets. 

Lines: ['1' '2' '3' '4' '5' '6' '7' 'S 42ND' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'JZ' 'L'
 'M' 'N' 'Q' 'R' 'S FKLN' 'S ROCK']
Divisions: ['A DIVISION' 'B DIVISION']
Day Types: ['1' '2']
Reporting Categories: ['CREW AVAILABILITY' 'EXTERNAL FACTORS' 'INFRASTRUCTURE & EQUIPMENT'
 'OPERATING CONDITIONS' 'PLANNED ROW WORK' 'POLICE & MEDICAL']


In [137]:
delayed_df

,month,division,line,day_type,incidents,reporting_category
1,2024-01-01,A DIVISION,1,1,31,CREW AVAILABILITY
2,2024-01-01,A DIVISION,1,1,2,EXTERNAL FACTORS
3,2024-01-01,A DIVISION,1,1,92,INFRASTRUCTURE & EQUIPMENT
4,2024-01-01,A DIVISION,1,1,86,OPERATING CONDITIONS
5,2024-01-01,A DIVISION,1,1,49,PLANNED ROW WORK
...,...,...,...,...,...,...
3761,2024-12-01,B DIVISION,S ROCK,2,16,EXTERNAL FACTORS
3762,2024-12-01,B DIVISION,S ROCK,2,6,INFRASTRUCTURE & EQUIPMENT
3763,2024-12-01,B DIVISION,S ROCK,2,1,OPERATING CONDITIONS
3764,2024-12-01,B DIVISION,S ROCK,2,1,PLANNED ROW WORK


**Dataset 2**

In [138]:
# returns json response - DATASET 2 "MTA Subway Customer Journey-Focused Metrics: 2020-2024”
results = client.get_all("4apg-4kt9", 
    where="month >= '2024-01' AND month <= '2024-12'")

In [139]:
# Convert to pandas DataFrame - DATASET 2 "MTA Subway Customer Journey-Focused Metrics: 2020-2024”
metrics_df = pd.DataFrame.from_records(results)
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time
0,2024-01-01T00:00:00.000,A DIVISION,1,offpeak,4873143.5,1.656052,0.79665726,8070179.0,3882225.2,763100.9,0.15659314,0.84340686
1,2024-01-01T00:00:00.000,A DIVISION,1,peak,4765373.0,2.115036,0.8733,10078935.0,4161600.2,743524.6,0.15602654,0.84397346
2,2024-01-01T00:00:00.000,A DIVISION,2,offpeak,2878163.8,1.6068327,0.50131786,4624727.5,1442874.9,487204.1,0.16927601,0.830724
3,2024-01-01T00:00:00.000,A DIVISION,2,peak,3092725.5,2.0733223,0.5617463,6412216.5,1737327.1,460791.2,0.14899196,0.85100806
4,2024-01-01T00:00:00.000,A DIVISION,3,offpeak,2125353.8,1.2491903,0.3281006,2654971.2,697329.8,265963.7,0.12513857,0.8748614
...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01T00:00:00.000,B DIVISION,S Fkln,peak,194189.34,0.5707303,0.057167325,110829.74,11101.285,9461.589,0.048723523,0.9512765
572,2024-12-01T00:00:00.000,B DIVISION,S Rock,offpeak,26513.352,1.2623241,-0.90806454,33468.44,-24075.834,3445.474,0.12995242,0.87004757
573,2024-12-01T00:00:00.000,B DIVISION,S Rock,peak,19257.482,0.8787027,-1.183044,16921.602,-22782.447,1812.6135,0.09412515,0.90587485
574,2024-12-01T00:00:00.000,B DIVISION,W,offpeak,1265461.6,0.9385583,0.2576316,1187709.5,326022.94,142330.02,0.112472795,0.8875272


**Dataset 2 - SHA256 Integrity Check**

In [140]:
# Write the data to disk as a CSV
metrics_df.to_csv("raw_data/metrics.csv", index=False)

# Read the CSV as bytes
with open("raw_data/metrics.csv", "rb") as f:
    data_bytes = f.read()

# Calculate SHA-256 of uncompressed data
sha256_hash = hashlib.sha256(data_bytes).hexdigest()

# Write the hex digest
with open("SHA256/metrics.sha", "w") as f:
    f.write(sha256_hash)

**Dataset 2 - Cleaning and Quality Assessment BEFORE INTEGRATION**

In [141]:
#Drop NaN Values
metrics_df = metrics_df.dropna()
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time
0,2024-01-01T00:00:00.000,A DIVISION,1,offpeak,4873143.5,1.656052,0.79665726,8070179.0,3882225.2,763100.9,0.15659314,0.84340686
1,2024-01-01T00:00:00.000,A DIVISION,1,peak,4765373.0,2.115036,0.8733,10078935.0,4161600.2,743524.6,0.15602654,0.84397346
2,2024-01-01T00:00:00.000,A DIVISION,2,offpeak,2878163.8,1.6068327,0.50131786,4624727.5,1442874.9,487204.1,0.16927601,0.830724
3,2024-01-01T00:00:00.000,A DIVISION,2,peak,3092725.5,2.0733223,0.5617463,6412216.5,1737327.1,460791.2,0.14899196,0.85100806
4,2024-01-01T00:00:00.000,A DIVISION,3,offpeak,2125353.8,1.2491903,0.3281006,2654971.2,697329.8,265963.7,0.12513857,0.8748614
...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01T00:00:00.000,B DIVISION,S Fkln,peak,194189.34,0.5707303,0.057167325,110829.74,11101.285,9461.589,0.048723523,0.9512765
572,2024-12-01T00:00:00.000,B DIVISION,S Rock,offpeak,26513.352,1.2623241,-0.90806454,33468.44,-24075.834,3445.474,0.12995242,0.87004757
573,2024-12-01T00:00:00.000,B DIVISION,S Rock,peak,19257.482,0.8787027,-1.183044,16921.602,-22782.447,1812.6135,0.09412515,0.90587485
574,2024-12-01T00:00:00.000,B DIVISION,W,offpeak,1265461.6,0.9385583,0.2576316,1187709.5,326022.94,142330.02,0.112472795,0.8875272


In [142]:
# Check data types
metrics_df.dtypes

month                       object
division                    object
line                        object
period                      object
num_passengers              object
additional_platform_time    object
additional_train_time       object
total_apt                   object
total_att                   object
over_five_mins              object
over_five_mins_perc         object
customer_journey_time       object
dtype: object

In [143]:
# Convert to correct data types
metrics_df['month'] = pd.to_datetime(metrics_df['month']).dt.normalize()
metrics_df['division'] = metrics_df['division'].astype(str)
metrics_df['line'] = metrics_df['line'].astype(str)
metrics_df['period'] = metrics_df['period'].astype(str)
metrics_df['num_passenger'] = metrics_df['num_passengers'].astype(float)
metrics_df['additional_platform_time'] = metrics_df['additional_platform_time'].astype(float)
metrics_df['additional_train_time'] = metrics_df['additional_train_time'].astype(float)
metrics_df['total_apt'] = metrics_df['total_apt'].astype(float)
metrics_df['total_att'] = metrics_df['total_att'].astype(float)
metrics_df['over_five_mins'] = metrics_df['over_five_mins'].astype(float)
metrics_df['over_five_mins_perc'] = metrics_df['over_five_mins_perc'].astype(float)
metrics_df['customer_journey_time'] = metrics_df['customer_journey_time'].astype(float)

In [144]:
#Clean string columns for synactic issues (capitalization and white space from front and back)
metrics_df['line'] = metrics_df['line'].str.strip().str.upper()
metrics_df['division'] = metrics_df['division'].str.strip().str.upper()
metrics_df['period'] = metrics_df['period'].str.strip().str.upper()

In [145]:
#Semantic check. Checking for negative times for numeric columns. 
cols_to_check = [
    'num_passenger', 'additional_platform_time', 'additional_train_time',
    'total_apt', 'total_att', 'over_five_mins',
    'over_five_mins_perc', 'customer_journey_time'
]

for col in cols_to_check:
    if (metrics_df[col] < 0).any():
        print(col)

additional_train_time
total_att


In [146]:
#Syntactic check, make sure all the months are formatted the same.
metrics_df['month'].dt.strftime('%Y-%m-%d').unique()

array(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
       '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
       '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
      dtype=object)

In [147]:
#Semantic check. Make sure all Division A are associated with numbered subway lines and 
#all Division B are associated with lettered subway lines

bad_A_2 = metrics_df[
    (metrics_df['division'] == 'A DIVISION') &
    ~metrics_df['line'].str.contains(r'\d', na=False)  
]
bad_B_2 = metrics_df[
    (metrics_df['division'] == 'B DIVISION') &
    ~metrics_df['line'].str.fullmatch(r'[A-Z ]+')  
]

print("A DIVISION with non-numbered lines:")
print(bad_A_2)

print("\nB DIVISION with non-lettered lines:")
print(bad_B_2)

A DIVISION with non-numbered lines:
Empty DataFrame
Columns: [month, division, line, period, num_passengers, additional_platform_time, additional_train_time, total_apt, total_att, over_five_mins, over_five_mins_perc, customer_journey_time, num_passenger]
Index: []

B DIVISION with non-lettered lines:
Empty DataFrame
Columns: [month, division, line, period, num_passengers, additional_platform_time, additional_train_time, total_apt, total_att, over_five_mins, over_five_mins_perc, customer_journey_time, num_passenger]
Index: []


In [148]:
#Check for duplicate rows across the entire dataset
metrics_df[metrics_df.duplicated()]

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time,num_passenger


In [149]:
# checking for syntactic and semantic errors syntax issues
print("Lines:", metrics_df['line'].unique())
print("Divisions:", metrics_df['division'].unique())
print("Period:", metrics_df['period'].unique())

Lines: ['1' '2' '3' '4' '5' '6' '7' 'S 42ND' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'JZ' 'L'
 'M' 'N' 'Q' 'R' 'S FKLN' 'S ROCK' 'W']
Divisions: ['A DIVISION' 'B DIVISION']
Period: ['OFFPEAK' 'PEAK']


In [150]:
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time,num_passenger
0,2024-01-01,A DIVISION,1,OFFPEAK,4873143.5,1.656052,0.796657,8.070179e+06,3882225.200,763100.9000,0.156593,0.843407,4873143.500
1,2024-01-01,A DIVISION,1,PEAK,4765373.0,2.115036,0.873300,1.007894e+07,4161600.200,743524.6000,0.156027,0.843973,4765373.000
2,2024-01-01,A DIVISION,2,OFFPEAK,2878163.8,1.606833,0.501318,4.624728e+06,1442874.900,487204.1000,0.169276,0.830724,2878163.800
3,2024-01-01,A DIVISION,2,PEAK,3092725.5,2.073322,0.561746,6.412216e+06,1737327.100,460791.2000,0.148992,0.851008,3092725.500
4,2024-01-01,A DIVISION,3,OFFPEAK,2125353.8,1.249190,0.328101,2.654971e+06,697329.800,265963.7000,0.125139,0.874861,2125353.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01,B DIVISION,S FKLN,PEAK,194189.34,0.570730,0.057167,1.108297e+05,11101.285,9461.5890,0.048724,0.951276,194189.340
572,2024-12-01,B DIVISION,S ROCK,OFFPEAK,26513.352,1.262324,-0.908065,3.346844e+04,-24075.834,3445.4740,0.129952,0.870048,26513.352
573,2024-12-01,B DIVISION,S ROCK,PEAK,19257.482,0.878703,-1.183044,1.692160e+04,-22782.447,1812.6135,0.094125,0.905875,19257.482
574,2024-12-01,B DIVISION,W,OFFPEAK,1265461.6,0.938558,0.257632,1.187710e+06,326022.940,142330.0200,0.112473,0.887527,1265461.600
